<a href="https://colab.research.google.com/github/Tavo826/UN/blob/main/Sol_EDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Organizar EDO solutions y búsqueda de raíces

In [ ]:
import matplotlib.style
matplotlib.style.use('classic')
import numpy as np
from matplotlib import pyplot as plt
from math import sin

%matplotlib inline

# Método de Euler

In [ ]:
def euler(func, h, tini, tfin, x0, params):

    '''
    ENTRADAS:

    func   -> función a evaluar
    h      -> tamaño del incremento
    tini   -> timepo inicial
    tfin   -> tiempo final
    x0     -> valor de la condición inicial
    params -> parámetros necesarios para el método

    SALIDAS:

    t -> lista de tiempos
    x -> lista de posiciones
    '''

    ti = tini  
    xi = x0
  
    x = list()
    t = list()
  
    while ( ti < tfin ):
    
      x.append (xi)
      t.append (ti)
  
      xi = xi + h*func(ti,xi,params)
      ti = ti + h
    return t, x 

# Método de Runge-Kutta orden 4

In [ ]:
def rk4(func, h, tini, tfin, x0, params):

    '''
    ENTRADAS:

    func   -> función a evaluar
    h      -> tamaño del incremento
    tini   -> timepo inicial
    tfin   -> tiempo final
    x0     -> valor de la condición inicial
    params -> parámetros necesarios para el método

    SALIDAS:

    t -> lista de tiempos
    x -> lista de posiciones
    '''

    ti = tini  
    xi = x0
  
    x = list()
    t = list()
  
    while ( ti < tfin ):

      x.append (xi)
      t.append (ti)
      k1 = func( ti,xi,params)
      k2 = func( ti + 0.5*h,xi + 0.5*k1*h,params )
      k3 = func( ti + 0.5*h,xi + 0.5*k2*h,params)
      k4 = func( ti + h,xi + k3*h,params) 

      xi = xi + h*(k1 + 2*k2 + 2*k3 + k4)/6
      ti = ti + h
    return t, x

# Método de Runge-Kutta adaptativo

In [ ]:
def odesolver12(f, t, y, h):
    """Calculate the next step of an initial value problem (IVP) of
    an ODE with a RHS described by f, with an order 1 approx.
    (Eulers Method) and an order 2 approx. (Midpoint Rule). This is 
    the simplest embedded RK pair.
    Parameters:
        f: function. RHS of ODE.
        t: float. Current time.
        y: float. Current position.
        h: float. Step length.
    Returns:
        q: float. Order 1 approx.
        w: float. Order 2 approx.
    """
    s1 = f(t, y)
    s2 = f(t+h, y+h*s1)
    w = y + h*s1
    q = y + h/2.0*(s1+s2)
    return w, q

def odesolver23(f, t, y, h):
    """Calculate the next step of an IVP of an ODE with a RHS
    described by f, with an order 2 approx. (Explicit Trapezoid 
    Method) and an order 3 approx. (third order RK).
    Parameters:
        f: function. RHS of ODE.
        t: float. Current time.
        y: float. Current position.
        h: float. Step length.
    Returns:
        q: float. Order 2 approx.
        w: float. Order 3 approx.
    """
    s1 = f(t, y)
    s2 = f(t+h, y+h*s1)
    s3 = f(t+h/2.0, y+h*(s1+s2)/4.0)
    w = y + h*(s1+s2)/2.0
    q = y + h*(s1+4.0*s3+s2)/6.0
    return w, q

def odesolver45(f, t, y, h):
    """Calculate the next step of an IVP of an ODE with a RHS
    described by f, with an order 4 approx. and an order 5 approx.
    Parameters:
        f: function. RHS of ODE.
        t: float. Current time.
        y: float. Current position.
        h: float. Step length.
    Returns:
        q: float. Order 2 approx.
        w: float. Order 3 approx.
    """
    s1 = f(t, y)
    s2 = f(t+h/4.0, y+h*s1/4.0)
    s3 = f(t+3.0*h/8.0, y+3.0*h*s1/32.0+9.0*h*s2/32.0)
    s4 = f(t+12.0*h/13.0, y+1932.0*h*s1/2197.0-7200.0*h*s2/2197.0+7296.0*h*s3/2197.0)
    s5 = f(t+h, y+439.0*h*s1/216.0-8.0*h*s2+3680.0*h*s3/513.0-845.0*h*s4/4104.0)
    s6 = f(t+h/2.0, y-8.0*h*s1/27.0+2*h*s2-3544.0*h*s3/2565+1859.0*h*s4/4104.0-11.0*h*s5/40.0)
    w = y + h*(25.0*s1/216.0+1408.0*s3/2565.0+2197.0*s4/4104.0-s5/5.0)
    q = y + h*(16.0*s1/135.0+6656.0*s3/12825.0+28561.0*s4/56430.0-9.0*s5/50.0+2.0*s6/55.0)
    return w, q

In [ ]:

def rk_adaptive(ode, rhs, y0=0.0, t0=0.0, TOL=1e-04, theta=1e-02, tmax=1.0):
    
    """Perform an adaptive RK method.
    Parameters:
        ode:   function. ODE solver.
        rhs:   function. RHS of ODE.
        y0:    float, optional. Initial position.
        t0:    float, optional. Initial time.
        TOL:   float, optional. Tolerance of relative error.
        theta: float, optional. "Protective" constant.
        tmax:  float, optional. End of calculation interval.
    Returns:
        y:     list. Position.
        t:     list. Time.
        i:     int. Number of iterations
    """
    
    # Allocate lists to store position and time and set
    # initial conditions.
    y = []
    t = []
    y.append(y0)
    t.append(t0)
    
    # Set initial step size and declare iteration integer
    h = 1.0
    i = 0
    
    while (t[i] < tmax):
        # Get two different approximations
        w, q = ode(rhs, t[i], y[i], h)
        # Estimate error
        e = abs((w-q)/max(w, theta))
        # If e larger thant TOL, decrease step length
        if (e > TOL):
            h = 0.8*(TOL*e)**(1/5)*h
            # Get two new approximations
            w, q = ode(rhs, t[i], y[i], h)
            # Estimate new error
            e = abs((w-q)/max(w, theta))
            # If e still larger than TOL, halve step length until False
            while (e > TOL):
                h = h/2.0
                # New approximations
                w, q = ode(rhs, t[i], y[i], h)
                # New error estimate
                e = abs((w-q)/max(w, theta))
        # Store highest order approximation as next y-value
        y.append(q)
        # Store current time + step size as next time
        t.append(t[i] + h)
        # Increment step number
        i += 1
        # Check if e is too small, if so, double step size
        if (e < 0.1*TOL):
            h = h*2.0
    
    return t, y, i

# Algoritmo de bisección

In [ ]:
x=np.linspace(-2,5,100)

y=np.exp(-x)-x
plt.plot(x,y)

def f(x): return np.exp(-x)-x
def df(x): return -np.exp(-x)-1
iteration=1

xi=-0.5
tol=1e-19
diff=np.abs(xi-x_next)

while tol<diff:
  x_next= xi-(f(xi)/df(xi))
  diff=np.abs(xi-x_next)
  xi=x_next
  iteration=iteration+1

plt.plot(xi,f(xi),'ro')
print('The root of f(x)=0 is %f with %f iterations' %(xi, iteration))

#Por iteraciones

N = 5

x_cur = 1.0
i=0
print(x_cur)

for i in range(N):
  i=i+1
  x_cur = x_cur-(2-x_cur**4-np.tanh(x_cur))/(-4*x_cur**3-1/(np.cosh(x_cur))**2)
  print(x_cur)
    
print(r'Value at last iteration, f(x_cur) = %s' % f(x_cur))

In [ ]:
# Algoritmo de Newton-Raphson

# Método de Euler explícito

# Método de Euler implícito

In [ ]:
def graficar(method, ri, rf, funcion, h, tini, tfin, params):

  '''
  ENTRADAS:

  method  -> método que se quiere graficar
  ri, rf  -> rango de condiciones iniciales que se desean evaluar
  funcion -> función que se desea evaluar
  tini    -> tiempo inicial
  tfin    -> tiempo final
  params  -> parámetros necesarios para el método

  SALIDA:
  
  Devuelve una figura cun el método evaluado en diferentes condiciones iniciales
  '''

  if method == 'euler' or method == 'Euler' or method == 'EULER':

    for x_0 in range(ri,rf):
      t1, x1 = euler(funcion, h, tini, tfin, x_0, params)
      if x_0 == ri:
        plt.scatter(t1, x1, c='red', s=30, label='Euler')
      else:
        plt.scatter(t1, x1, c='red', s=30)

  if method == 'RK4' or method == 'rk4' or method == 'Rk4':
    for x_0 in range(ri,rf):
      t2, x2 = rk4(funcion, h, tini, tfin, x_0, params)
      if x_0 == ri:
        plt.scatter(t2, x2, c='blue', s=20, label="RK4")
      else:
        plt.scatter(t2, x2, c='blue', s=20)

  if method == 'rk_adaptarive' or method == 'rk_adaptativo':
    

  plt.ylabel('x(t)')
  plt.xlabel(r'$t$')
  plt.grid()
  plt.legend(loc='best')
  plt.show()